## Introduction

The Wallaroo MLOps API allows organizations to submit requests to their Wallaroo instance to perform such actions as:

* Create a new user and invite them to the instance.
* Create workspaces and list their configuration details.
* Upload a model.
* Deploy and undeploy a pipeline.

The following examples will show how to submit queries to the Wallaroo MLOps API and the types of responses returned.

### References

The following references are available for more information about Wallaroo and the Wallaroo MLOps API:

* [Wallaroo Documentation Site](https://docs.wallaroo.ai):  The Wallaroo Documentation Site
* Wallaroo MLOps API Documentation from a Wallaroo instance:  A Swagger UI based documentation is available from your Wallaroo instance at `https://{Wallaroo Prefix}.api.{Wallaroo Suffix}/v1/api/docs`.  For example, if the Wallaroo Instance is Wallaroo Community with the prefix `{lovely-rhino-5555}`, then the Wallaroo MLOps API Documentation would be available at `https://lovely-rhino-5555.api.wallaroo.community/v1/api/docs`.  For another example, a Wallaroo Enterprise users who do not use a prefix and has the suffix `wallaroo.example.com`, the the Wallaroo MLOps API Documentation would be available at `https://api.wallaroo.example.com/v1/api/docs`.  For more information, see the [Wallaroo Documentation Site](https://docs.wallaroo.ai).

**IMPORTANT NOTE**:  This tutorial is in beta while the MLOps API is updated.  Some commands may not work as currently documented and will be updated in an upcoming release.  Other commands may undergo changes to be more user friendly.

## OpenAPI Steps

The following demonstrates how to use each command in the Wallaroo MLOps API, and can be modified as best fits your organization's needs.

### Import Libraries

For the examples, the Python `requests` library will be used.

In [1]:
import requests
from requests.auth import HTTPBasicAuth

### Set Variables

The following variables are used for the example and should be modified to fit your organization.

Wallaroo comes pre-installed with a confidential OpenID Connect client.  The default client is `api-client`, but other clients may be created and configured.

As it is a confidential client, api-client requires its secret to be supplied when requesting a token. Administrators may obtain their API client credentials from Keycloak from the Keycloak Service URL as listed above and the prefix `/auth/admin/master/console/#/realms/master/clients`.

For example, if the Wallaroo Community instance DNS address is `https://magical-rhino-5555.wallaroo.community`, then the direct path to the Keycloak API client credentials would be:

`https://magical-rhino-5555.keycloak.wallaroo.community/auth/admin/master/console/#/realms/master/clients`

Then select the client, in this case **api-client**, then **Credentials**.

![Wallaroo Keycloak Service](./images/wallaroo-api/wallaroo-api-keycloak-service.png)

![Wallaroo Components](./images/wallaroo-api/wallaroo-api-keycloak-credentials.png)

By default, tokens issued for api-client are valid for up to 60 minutes. Refresh tokens are supported.

In [2]:
## Variables

URLPREFIX='magical-bear-3782'
URLSUFFIX='wallaroo.community'
TOKENURL=f'https://{URLPREFIX}.keycloak.{URLSUFFIX}/auth/realms/master/protocol/openid-connect/token'
CLIENT="api-client"
SECRET="yoursecrethere"
USERNAME="your user email here"
PASSWORD="insertyourpasswordhere"
APIURL=f"https://{URLPREFIX}.api.{URLSUFFIX}/v1/api"
newUser="your new user email here"
newPassword="new user password here"

The following is an output of the `TOKENURL` variable to verify it matches your Wallaroo instance's Keycloak API client credentials URL.

In [3]:
TOKENURL

'https://magical-bear-3782.keycloak.wallaroo.community/auth/realms/master/protocol/openid-connect/token'

### API Example Methods

The following methods are used to retrieve the MLOPs API Token from the Wallaroo instance's Keycloak service, and submit MLOps API requests through the Wallaroo instance's MLOps API.

MLOps API requests are always `POST`, and are either submitted as `'Content-Type':'application/json'` or as a multipart submission including a file.

In [4]:
def get_jwt_token(url, client, secret, username, password):
    auth = HTTPBasicAuth(client, secret)
    data = {
        'grant_type': 'password',
        'username': username,
        'password': password
    }
    response = requests.post(url, auth=auth, data=data, verify=True)
    return response.json()['access_token']


# This can either submit a plain POST request ('Content-Type':'application/json'), or with a file.

def get_wallaroo_response(url, api_request, token, data, files=None):
    apiUrl=f"{url}{api_request}"
    if files is None:
        # Regular POST request
        headers= {
            'Authorization': 'Bearer ' + token,
            'Content-Type':'application/json'
        }
        response = requests.post(apiUrl, json=data, headers=headers, verify=True)
    else:
        # POST request with file
        headers= {
            'Authorization': 'Bearer ' + token
        }
        response = requests.post(apiUrl, data=data, headers=headers, files=files, verify=True)
    return response.json()

### Retrieve MLOps API Token

To retrieve an API token for a specific user with the Client Secret, request the token from the Wallaroo instance using the client secret and provide the following:

* Token Request URL: The Keycloak token retrieval URL.
* OpenID Connect client name: The name of the OpenID Connect client.
* OpenID Connect client Secret:  The secret for the OpenID Connect client.
* UserName:  The username of the Wallaroo instance user, usually their email address.
* Password:  The password of the Wallaroo instance user.

The following sample uses the variables set above to request the token, then displays it.

In [5]:
TOKEN=get_jwt_token(TOKENURL, CLIENT, SECRET, USERNAME, PASSWORD)

In [6]:
TOKEN

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ1UTNmY2xMTlktWjREcVJhbmxlczMwZU1wUjJaR1pnVlhfMVlkTWJjRmdrIn0.eyJleHAiOjE2NjM2OTg2MDUsImlhdCI6MTY2MzY5NTAwNSwianRpIjoiMjg2NmU1MjgtYzEwYy00ZWQwLWExMDAtMzI2ODk3MDZlNDk2IiwiaXNzIjoiaHR0cHM6Ly9tYWdpY2FsLWJlYXItMzc4Mi5rZXljbG9hay53YWxsYXJvby5jb21tdW5pdHkvYXV0aC9yZWFsbXMvbWFzdGVyIiwiYXVkIjpbIm1hc3Rlci1yZWFsbSIsImFjY291bnQiXSwic3ViIjoiMzI0MDlhYmYtZjkyNy00ZDFhLWEzZDctNjdmZmJmMTc3ZDlmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYXBpLWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiI4MzIxOTA0Zi1lZWYxLTRlZDAtODE0ZS1mZjJkODJhMzcwNzEiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbIioiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImNyZWF0ZS1yZWFsbSIsImRlZmF1bHQtcm9sZXMtbWFzdGVyIiwib2ZmbGluZV9hY2Nlc3MiLCJhZG1pbiIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsibWFzdGVyLXJlYWxtIjp7InJvbGVzIjpbInZpZXctcmVhbG0iLCJ2aWV3LWlkZW50aXR5LXByb3ZpZGVycyIsIm1hbmFnZS1pZGVudGl0eS1wcm92aWRlcnMiLCJpbXBlcnNvbmF0aW9uIiwiY3JlYXRlLWNsaWVudCIsIm1hbmFnZS11c2VycyIsInF1ZXJ5LXJlYWxtcyIsInZpZXctYXV0aG9yaXphdGlvbiI

### Get Users

Users can be retrieved either by their Keycloak user id, or return all users if an empty set `{}` is submitted.

* **Parameters**
  * **user_ids** - (OPTIONAL *Array[Keycloak user ids]*): An array of Keycloak user ids, typically in UUID format.  If an empty set `{}` is submitted then all users are returned.
* **Returns**
  * **users** (List user): List of Wallaroo users listed with their Keycloak ID as the JSON key.  Each user contains the following attributes.
    * **id** (string): Keycloak id of the user.
    * **createdTimestamp**': 1663694386025,
    * **username** (string): Username of the user, usually their email address.
    * **enabled** (Bool): Whether the user is enabled or not.
    * **totp** (Bool): Whether to use Time Based (TOTP) and Counter Based (HOTP) for the One Time Password (OTP).
    * **emailVerified** (Bool): Whether the user's email address has been verified.
    * **email** (string): The user's email address.
    * **disableableCredentialTypes** (Array): 
    * **requiredActions** (Array): Actions the users must complete before they can log in.
    * **notBefore**
    * **access** (object): Levels of access the user is granted.
    *   * **manageGroupMembership** (Bool)
        * **view** (Bool)
        * **mapRoles** (Bool)
        * **impersonate** (Bool)
        * **manage** (Bool)

Example:  The first example will submit an empty set `{}` to return all users, then submit the first user's user id and request only that user's details.

In [7]:
# Get all users

apiRequest = "/users/query"
data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'users': {'32409abf-f927-4d1a-a3d7-67ffbf177d9f': {'id': '32409abf-f927-4d1a-a3d7-67ffbf177d9f',
   'createdTimestamp': 1663694386025,
   'username': 'john.hansarick@wallaroo.ai',
   'enabled': True,
   'totp': False,
   'emailVerified': True,
   'email': 'john.hansarick@wallaroo.ai',
   'disableableCredentialTypes': [],
   'requiredActions': [],
   'notBefore': 0,
   'access': {'manageGroupMembership': True,
    'view': True,
    'mapRoles': True,
    'impersonate': True,
    'manage': True}}}}

In [8]:
# Get first user Keycloak id
firstUserKeycloak = list(response['users'])[0]

apiRequest = "/users/query"
data = {
  "user_ids": [
    firstUserKeycloak
  ]
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'users': {'32409abf-f927-4d1a-a3d7-67ffbf177d9f': {'id': '32409abf-f927-4d1a-a3d7-67ffbf177d9f',
   'createdTimestamp': 1663694386025,
   'username': 'john.hansarick@wallaroo.ai',
   'enabled': True,
   'totp': False,
   'emailVerified': True,
   'email': 'john.hansarick@wallaroo.ai',
   'disableableCredentialTypes': [],
   'requiredActions': [],
   'notBefore': 0,
   'access': {'manageGroupMembership': True,
    'view': True,
    'mapRoles': True,
    'impersonate': True,
    'manage': True}}}}

### Invite Users

Users can be invited through `/users/invite`.

* **Parameters**
  * **email** (*REQUIRED string*): The email address of the new user to invite.
  * **password** (*OPTIONAL string*): The assigned password of the new user to invite.  If not provided, the Wallaroo instance will provide the new user a temporary password that must be changed upon initial login.
* **Returns**
  * **id** (string): The Keycloak id of the new user.
  * **password** (string):  The password of the new user **if** a password was specified during creation.

Example:  In this example, a new user will be invited to the Wallaroo instance and assigned a password.

In [9]:
# invite users
apiRequest = "/users/invite"
data = {
    "email": newUser,
    "password":newPassword
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'id': '0aba1f11-7736-48f9-b81b-fce71ab33f98', 'password': 'Snugglebunnies?'}

### Deactivate User

Users can be deactivated so they can not login to their Wallaroo instance.  Deactivated users do not count against the Wallaroo license count.

* **Parameters**
  * **email** (*REQUIRED string*):  The email address of the user to deactivate.

**NOTE:** Disabled for now until further testing is done.

In [10]:
# Deactivate users

# apiRequest = "/users/deactivate"

#data = {
#    "email": "kilvin.mitchell@wallaroo.ai"
#}

#response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
#response

### Activate User

A deactivated user can be reactivated to allow them access to their Wallaroo instance.  Activated users count against the Wallaroo license count.

* **Parameters**
  * **email** (*REQUIRED string*):  The email address of the user to activate.
  
**NOTE:** Disabled for now until further testing is done.

In [11]:
# Activate users

#apiRequest = "/users/activate"

#data = {
#    "email": "john.hummel@wallaroo.ai"
#}

#response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)

### List Workspaces

List the workspaces for a specific user.

* **Parameters**
  * **user_id** - (*OPTIONAL string*): The email address of the user.
  
Example:  In this example, the workspaces for the a specific user will be displayed, then workspaces for all users will be displayed.

**NOTE**:  In Mustang release there is a bug that still shows all workspaces whether a `user_id` is supplied or not.

In [12]:
# List workspaces by user id

apiRequest = "/workspaces/list"

data = {
    "user_id":newUser
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'workspaces': [{'id': 1,
   'name': 'john.hansarick@wallaroo.ai - Default Workspace',
   'created_at': '2022-09-20T17:19:46.625381+00:00',
   'created_by': '32409abf-f927-4d1a-a3d7-67ffbf177d9f',
   'archived': False,
   'models': [],
   'pipelines': []}]}

In [13]:
# List workspaces

apiRequest = "/workspaces/list"

data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'workspaces': [{'id': 1,
   'name': 'john.hansarick@wallaroo.ai - Default Workspace',
   'created_at': '2022-09-20T17:19:46.625381+00:00',
   'created_by': '32409abf-f927-4d1a-a3d7-67ffbf177d9f',
   'archived': False,
   'models': [],
   'pipelines': []}]}

### Create Workspace

A new workspace will be created in the Wallaroo instance.  Upon creating, it's owner will be assigned as the user making the MLOps API request.

* **Parameters**:
  * **workspace_name** - (*REQUIRED string*):  The name of the new workspace.
* **Returns**:
  * **workspace_id** - (*int*):  The ID of the new workspace.
  
Example:  In this example, a workspace with the name `ccfraudapi` will be created, and it's `workspace_id` saved for use in other code examples.  After the request is complete, the List Workspaces command will be issued to demonstrate the new workspace has been created.

In [14]:
# Create workspace

apiRequest = "/workspaces/create"

exampleWorkspaceName = "testapiworkspace"
data = {
  "workspace_name": exampleWorkspaceName
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleWorkspaceId = response['workspace_id']
response

{'workspace_id': 3}

### Add User to Workspace

Existing users of the Wallaroo instance can be added to an existing workspace.

* **Parameters**
  * **email** - (*REQUIRED string*):  The email address of the user to add to the workspace.
  * **workspace_id** - (*REQUIRED int*): The id of the workspace.
  
Example:  The following example adds the user created in Invite Users request to the workspace created in the Create Workspace request.

In [15]:
# Add existing user to existing workspace

apiRequest = "/workspaces/add_user"

data = {
  "email":newUser,
  "workspace_id": exampleWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{}

### List Workspaces

Lists of workspaces either for all or by user_id.

* **Paramters**
  * **user_id** - (*OPTIONAL string*): The email address of the specific user to request workspaces for.
* **Returns**
  * **users** - (Array)
    * **user_id** -  (*string*): The email address of the user.
    * **user_type**" (*string*): The type of user, either `co-owner` or `collaborator`.

Example:  An empty set will be submitted to return all workspaces and their users, then only the user added to a workspace from the step Add User to Workspace.

**IMPORTANT NOTE**:  The filter by `user_id` currently is not working as documented, and will be address in a future release.

In [16]:
# List all workspaces

apiRequest = "/workspaces/list"

data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'workspaces': [{'id': 1,
   'name': 'john.hansarick@wallaroo.ai - Default Workspace',
   'created_at': '2022-09-20T17:19:46.625381+00:00',
   'created_by': '32409abf-f927-4d1a-a3d7-67ffbf177d9f',
   'archived': False,
   'models': [],
   'pipelines': []},
  {'id': 3,
   'name': 'testapiworkspace',
   'created_at': '2022-09-20T17:30:07.656704+00:00',
   'created_by': '32409abf-f927-4d1a-a3d7-67ffbf177d9f',
   'archived': False,
   'models': [],
   'pipelines': []}]}

In [17]:
# List specific workspace

apiRequest = "/workspaces/list"

data = {
    "user_id":newUser
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'workspaces': [{'id': 1,
   'name': 'john.hansarick@wallaroo.ai - Default Workspace',
   'created_at': '2022-09-20T17:19:46.625381+00:00',
   'created_by': '32409abf-f927-4d1a-a3d7-67ffbf177d9f',
   'archived': False,
   'models': [],
   'pipelines': []},
  {'id': 3,
   'name': 'testapiworkspace',
   'created_at': '2022-09-20T17:30:07.656704+00:00',
   'created_by': '32409abf-f927-4d1a-a3d7-67ffbf177d9f',
   'archived': False,
   'models': [],
   'pipelines': []}]}

### Upload Model to Workspace

Uploads a ML Model to a Wallaroo workspace as a POST with `Content-Type: multipart/form-data`.

* **Parameters**
  * **name** - (*REQUIRED string*): Name of the model
  * **visibility** - (*OPTIONAL string*): The visibility of the model as either `public` or `private`.
  * **workspace_id** - (*REQUIRED int*): The numerical id of the workspace to upload the model to.
  
Example:  This example will upload the sample file `ccfraud.onnx` to the workspace created in the Create Workspace step as `apitestmodel`.  The id of the uploaded model will be saved for later examples.

In [18]:
# upload model - have a multiform data through Python

apiRequest = "/models/upload"

exampleModelName = "apitestmodel"

data = {
    "name":exampleModelName,
    "visibility":"public",
    "workspace_id": exampleWorkspaceId
}

files = {
    "file": ('ccfraud.onnx', open('ccfraud.onnx','rb'))
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data, files)
response

{'insert_models': {'returning': [{'models': [{'id': 1}]}]}}

In [19]:
exampleModelId=response['insert_models']['returning'][0]['models'][0]['id']
exampleModelId

1

### List Models in Workspace

Returns a list of models added to a specific workspace.

* **Parameters**
  * **workspace_id** - (*REQUIRED int*): The workspace id to list.
  
Example:  Display the models for the workspace used in the Upload Model to Workspace step.  The details of the models will be saved as variables for other examples.

In [20]:
# List models in a workspace

apiRequest = "/models/list"

data = {
  "workspace_id": exampleWorkspaceId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'models': [{'id': 1,
   'models': {'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
    'models_pk_id': 1,
    'model_version': 'fe1ad4de-6b1c-4d73-99a9-fe19e276115a',
    'owner_id': '""',
    'model_id': 'apitestmodel',
    'id': 1,
    'file_name': 'ccfraud.onnx',
    'image_path': None}}]}

In [21]:
exampleModelSha = response['models'][0]['models']['sha']
exampleModelVersion = response['models'][0]['models']['model_version']

### Get Model Details

Returns the model details by the specific model id.

* **Parameters**
  * **workspace_id** - (*REQUIRED int*): The workspace id to list.
* **Returns**
  * **id** - (*int*):  Numerical id of the model.
  * **owner_id** - (*string*): Id of the owner of the model.
  * **workspace_id** - (*int*): Numberical of the id the model is in.
  * **name** - (*string*): Name of the model.
  * **updated_at** - (*DateTime*): Date and time of the model's last update.
  * **created_at** - (*DateTime*): Date and time of the model's creation.
  * **model_config** - (*string*): Details of the model's configuration.
  
Example:  Retrieve the details for the model uploaded in the Upload Model to Workspace step.

In [22]:
# Get model details by id

apiRequest = "/models/get_by_id"

data = {
  "id": exampleModelId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'id': 1,
 'owner_id': '""',
 'workspace_id': 3,
 'name': 'apitestmodel',
 'updated_at': '2022-09-20T17:30:09.368426+00:00',
 'created_at': '2022-09-20T17:30:09.368426+00:00',
 'model_config': None}

### Get Model Configuration by Id

Returns the model's configuration details.

* **Parameters**
  * **model_id** - (*REQUIRED int*): The numerical value of the model's id.
  
Example:  Submit the model id for the model uploaded in the Upload Model to Workspace step.

In [23]:
# Get model config by id

apiRequest = "/models/get_config_by_id"

data = {
  "model_id": exampleModelId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'model_config': None}

### Create Pipeline in a Workspace

Creates a new pipeline in the specified workspace.

* **Parameters**
  * **pipeline_id** - (REQUIRED string): Name of the new pipeline.
  * **workspace_id** - (REQUIRED int): Numberical id of the workspace for the new pipeline.
  * **definition** - (REQUIRED string): Pipeline definitions, can be `{}` for none.

Example:  Two pipelines are created in the workspace created in the step Create Workspace.  One will be an empty pipeline without any models, the other will be created using the uploaded models in the Upload Model to Workspace step and no configuration details.  The pipeline details will be stored for later examples.

In [27]:
# Create pipeline in a workspace

apiRequest = "/pipelines/create"

emptyPipelineId="emptypipeline"

data = {
  "pipeline_id": emptyPipelineId,
  "workspace_id": exampleWorkspaceId,
  "definition": {}
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleEmptyPipelineId = response['pipeline_pk_id']
exampleEmptyPipelineVariantId=response['pipeline_variant_pk_id']
emptyExamplePipelineVariantVersion=['pipeline_variant_version']
response

{'pipeline_pk_id': 3,
 'pipeline_variant_pk_id': 3,
 'pipeline_variant_version': 'e88776c0-0b7e-468f-be3a-7fc86c1efd50'}

In [25]:
response

{'msg': 'A pipeline was created but variant creation failed', 'code': 500}

In [26]:
# Create pipeline in a workspace with models

apiRequest = "/pipelines/create"

modelPipelineId="pipelinewithmodel"

data = {
  "pipeline_id": modelPipelineId,
  "workspace_id": exampleWorkspaceId,
  "definition": {
      "id":"ccfrauddeploytest",
      "steps":
      [
          {
          "ModelInference":
          {
              "models": [
                    {
                        "name":exampleModelName,
                        "version":exampleModelVersion,
                        "sha":exampleModelSha
                    }
                ]
          }
          }
      ]
  }
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleModelPipelineId = response['pipeline_pk_id']
exampleModelPipelineVariantId=response['pipeline_variant_pk_id']
emptyModelPipelineVariantVersion=['pipeline_variant_version']
response

{'pipeline_pk_id': 2,
 'pipeline_variant_pk_id': 2,
 'pipeline_variant_version': 'cfa11514-2c35-4fad-bd2b-251b10daca79'}

### Deploy a Pipeline

Deploy a an existing pipeline.  Note that for any pipeline that has model steps, they must be included either in `model_configs`, `model_ids` or `models`.

* **Parameters**
  * **deploy_id** (*REQUIRED string*): The name for the pipeline deployment.
  * **engine_config** (*OPTIONAL string*): Additional configuration options for the pipeline.
  * **pipeline_version_pk_id** (*REQUIRED int*): Pipeline version id.
  * **model_configs** (*OPTIONALArray int*): Ids of model configs to apply.
  * **model_ids** (*OPTIONALArray int*): Ids of models to apply to the pipeline.  If passed in, model_configs will be created automatically.
  * **models** (*OPTIONAL Array models*):  If the model ids are not available as a pipeline step, the models' data can be passed to it through this method.  The options below are only required if `models` are provided as a parameter.
    * **name** (*REQUIRED string*): Name of the uploaded model that is in the same workspace as the pipeline.
    * **version** (*REQUIRED string*): Version of the model to use.
    * **sha** (*REQUIRED string*): SHA value of the model.
  * **pipeline_id** (*REQUIRED int*): Numerical value of the pipeline to deploy.
* **Returns**
  * **id** (*int*): The deployment id.

Examples:  Both the empty pipeline and pipeline with model created in the step Create Pipeline in a Workspace will be deployed and their deployment information saved for later examples.


In [29]:
# Deploy empty pipeline

apiRequest = "/pipelines/deploy"

exampleEmptyDeployId = "emptydeploy"

data = {
    "deploy_id": exampleEmptyDeployId,
    "pipeline_version_pk_id": exampleEmptyPipelineVariantId,
    "pipeline_id": exampleEmptyPipelineId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleEmptyDeploymentId=response['id']
response


{'id': 1}

In [30]:
# Deploy a pipeline with models

apiRequest = "/pipelines/deploy"
exampleModelDeployId="modeldeploy"

data = {
    "deploy_id": exampleModelDeployId,
    "pipeline_version_pk_id": exampleModelPipelineVariantId,
    "models": [
        {
            "name":exampleModelName,
            "version":exampleModelVersion,
            "sha":exampleModelSha
        }
    ],
    "pipeline_id": exampleModelPipelineId
}


response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
exampleModelDeploymentId=response['id']
response

{'id': 2}

## Get Deployment Status

Returns the deployment status.

* **Parameters**
  * **name** - (REQUIRED string): The deployment in the format {deployment_name}-{deploymnent-id}.
  
Example: The deployed empty and model pipelines status will be displayed.

In [31]:
# Get empty pipeline deployment

apiRequest = "/status/get_deployment"

data = {
  "name": f"{exampleEmptyDeployId}-{exampleEmptyDeploymentId}"
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'status': 'Running',
 'details': None,
 'engines': [{'ip': '10.244.2.24',
   'name': 'engine-6d57d66b4c-ssn6p',
   'status': 'Running',
   'reason': None,
   'pipeline_statuses': {'pipelines': [{'id': 'emptypipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': []}}],
 'engine_lbs': [{'ip': '10.244.2.25',
   'name': 'engine-lb-67c854cc86-ffn4w',
   'status': 'Running',
   'reason': None}]}

In [32]:
# Get model pipeline deployment

apiRequest = "/status/get_deployment"

data = {
  "name": f"{exampleModelDeployId}-{exampleModelDeploymentId}"
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'status': 'Running',
 'details': None,
 'engines': [{'ip': '10.244.1.25',
   'name': 'engine-6cc4cdc9db-rblzt',
   'status': 'Running',
   'reason': None,
   'pipeline_statuses': {'pipelines': [{'id': 'pipelinewithmodel',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'apitestmodel',
      'version': 'fe1ad4de-6b1c-4d73-99a9-fe19e276115a',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.1.24',
   'name': 'engine-lb-67c854cc86-tqst9',
   'status': 'Running',
   'reason': None}]}

### Undeploy a Pipeline

Undeploys a deployed pipeline.

* **Parameters**
  * **pipeline_id** - (*REQUIRED int*): The numerical id of the pipeline.
  * **deployment_id** - (*REQUIRED int*): The numerical id of the deployment.
* **Returns**
  * Nothing if the call is successful.

Example:  Both the empty pipeline and pipeline with models deployed in the step Deploy a Pipeline will be undeployed.

In [34]:
# Undeploy an empty pipeline

apiRequest = "/pipelines/undeploy"

data = {
    "pipeline_id": exampleEmptyPipelineId,
    "deployment_id":exampleEmptyDeploymentId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

In [35]:
# Undeploy pipeline with models

apiRequest = "/pipelines/undeploy"

data = {
    "pipeline_id": exampleModelPipelineId,
    "deployment_id":exampleModelDeploymentId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

### Copy a Pipeline

Copies an existing pipeline into a new one in the same workspace.  A new engine configuration can be set for the copied pipeline.

* **Parameters**
  * **name** - (REQUIRED string): The name of the new pipeline.
  * **workspace_id** - (REQUIRED int): The numerical id of the workspace to copy the source pipeline from.
  * **source_pipeline** - (REQUIRED int): The numerical id of the pipeline to copy from.
  * **deploy** - (OPTIONAL string): Name of the deployment.
  * **engine_config** - (OPTIONAL string): Engine configuration options.
  * **pipeline_version** - (OPTIONAL string): Optional version of the copied pipeline to create.

Example:  The pipeline with models created in the step Create Pipeline in a Workspace will be copied into a new one.


In [36]:
# Copy a pipeline

apiRequest = "/pipelines/copy"

exampleCopiedPipelineName="copiedmodelpipeline"

data = {
  "name": exampleCopiedPipelineName,
  "workspace_id": exampleWorkspaceId,
  "source_pipeline": exampleModelPipelineId
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'pipeline_pk_id': 4,
 'pipeline_variant_pk_id': 4,
 'pipeline_version': None,
 'deployment': None}

## List Enablement Features

Lists the enablemane features for the Wallaroo instance.

* **PARAMETERS**
  * null:  An empty set `{}`
* **RETURNS**
  * **features** - (*string*): Enabled features.
  * **name** - (*string*): Name of the Wallaroo instance.
  * **is_auth_enabled** - (*bool*): Whether authentication is enabled.

In [37]:
# List enablement features

apiRequest = "/features/list"

data = {
}

response = get_wallaroo_response(APIURL, apiRequest, TOKEN, data)
response

{'features': {'plateau': 'true'},
 'name': 'Wallaroo Dev',
 'is_auth_enabled': True}